# Notebook for creating a data frame that's needed for the dashboard

In [203]:
import numpy as np
import pandas as pd
import plotly.express as px
import plotly.graph_objects as go
import plotly.express as px
from io import SEEK_CUR
import pandas as pd
import numpy as np

import dash
from datetime import date
import dash_bootstrap_components as dbc
from dash import dcc
from dash import html
import plotly.graph_objects as go
import plotly.express as px

from PIL import ImageColor

from dash.dependencies import Input, Output, State

from dashboard_main import *

In [204]:
features_forecast = ['ZONEID', 'TIMESTAMP', 'HOUR', 'WEEKDAY', 'WS100', 'WD100']
features_retro = ['ZONEID', 'TIMESTAMP', 'HOUR', 'WEEKDAY', 'TARGETVAR']

PATH_PREDICTIONS = 'RandomForest_Predictions.csv'
PATH_DATA_ALL = 'raw_data_incl_features_test.csv'

initial_date = '2013-07-01'

In [205]:
## load the data
data_forecast = pd.read_csv(PATH_PREDICTIONS, parse_dates=['TIMESTAMP'])
data_forecast.rename(
    columns={x: 'Wind Farm '+x.split()[-1] 
    for x in data_forecast.columns if x.startswith('Zone')}, 
    inplace=True)
add_HOUR_column(data_forecast)
data_all = pd.read_csv(PATH_DATA_ALL, parse_dates=['TIMESTAMP'])
data_all.interpolate(method='ffill', inplace=True)
data_all['ZONEID'] = data_all['ZONEID'].apply(lambda x: 'Wind Farm '+str(x))
data_all['HOUR'].replace(0,24, inplace=True)
data_all.replace('Zone', 'Wind Farm', inplace=True)
data_all.reset_index(inplace=True)

## First the dataframe for the forecasted values

In [206]:
# To ensure the the grouping by day is correct, substract one minute from the timestamp
# Then a date column is created and the whole dataframe grouped by the date 
delta_minute = pd.Timedelta(1, unit='min')
data_forecast['TIMESTAMP'] = data_forecast['TIMESTAMP'].apply(lambda x: x-delta_minute)
data_forecast.drop('HOUR', axis=1, inplace=True)
data_forecast['DATE'] = data_forecast['TIMESTAMP'].dt.date 
data_forecast = data_forecast.groupby('DATE').mean()
#data_forecast.head()

In [207]:
## Bring the dataframe into a long form
data_forecast_rsi = data_forecast.reset_index()
data_forecast_long = pd.wide_to_long(data_forecast_rsi, stubnames='Wind Farm ', i='DATE', j='ZONEID')
data_forecast_long.reset_index('ZONEID', inplace=True)
data_forecast_long.columns = ['ZONEID', 'TARGETVAR_PREDICTED']
data_forecast_long['ZONEID'] = data_forecast_long['ZONEID'].apply(lambda x: 'Wind Farm '+str(x))
data_forecast_long

,ZONEID,TARGETVAR_PREDICTED
DATE,,
2013-07-01,Wind Farm 1,0.489216
2013-07-02,Wind Farm 1,0.466056
2013-07-03,Wind Farm 1,0.660869
2013-07-04,Wind Farm 1,0.781130
2013-07-05,Wind Farm 1,0.673151
...,...,...
2013-12-27,Wind Farm 10,0.454930
2013-12-28,Wind Farm 10,0.805684
2013-12-29,Wind Farm 10,0.498067


## observed values

In [208]:
## observed values
data_all['TIMESTAMP'] = data_all['TIMESTAMP'].apply(lambda x: x-delta_minute)
data_all['DATE'] = data_all['TIMESTAMP'].dt.date

In [209]:
data_all_mean = data_all.groupby(['DATE', 'ZONEID']).mean().reset_index('ZONEID')
data_all_mean = data_all_mean[['ZONEID','TARGETVAR']]
data_all_mean.columns = ['ZONEID', 'TARGETVAR_OBSERVED']
data_all_mean

,ZONEID,TARGETVAR_OBSERVED
DATE,,
2013-06-30,Wind Farm 1,0.593534
2013-06-30,Wind Farm 10,0.243962
2013-06-30,Wind Farm 2,0.375511
2013-06-30,Wind Farm 3,0.150179
2013-06-30,Wind Farm 4,0.265700
...,...,...
2013-12-31,Wind Farm 5,0.313211
2013-12-31,Wind Farm 6,0.487690
2013-12-31,Wind Farm 7,0.353346


## combining the forecasted and the observed energy production

In [210]:
df_comparison = data_all_mean.merge(data_forecast_long, how='inner', on=['DATE', 'ZONEID'])
df_comparison.head()

,ZONEID,TARGETVAR_OBSERVED,TARGETVAR_PREDICTED
DATE,,,
2013-07-01,Wind Farm 1,0.729795,0.489216
2013-07-01,Wind Farm 10,0.181076,0.255032
2013-07-01,Wind Farm 2,0.385629,0.296637
2013-07-01,Wind Farm 3,0.242851,0.290418
2013-07-01,Wind Farm 4,0.137524,0.182541


In [216]:
fig = go.Figure()

df_comparison_zone = df_comparison[df_comparison['ZONEID'] == 'Wind Farm 3']

fig.add_traces(
    go.Scatter(x=df_comparison_zone.index, y = df_comparison_zone['TARGETVAR_OBSERVED'], 
        mode = 'lines', line=dict(color='red',), name='observed',
    )
)

fig.add_traces(
    go.Scatter(x=df_comparison_zone.index, y = df_comparison_zone['TARGETVAR_PREDICTED'], 
        mode = 'lines', line=dict(color='blue',), name='predicted',
    )
)

In [199]:
## Saving the comparison dataframe for use on dashboard
df_comparison.to_csv('data_comparison_predicted_observed.csv', index=True)
df_comparison.head()

,ZONEID,TARGETVAR_OBSERVED,TARGETVAR_PREDICTED
DATE,,,
2013-07-01,Wind Farm 1,0.729795,0.489216
2013-07-01,Wind Farm 10,0.181076,0.255032
2013-07-01,Wind Farm 2,0.385629,0.296637
2013-07-01,Wind Farm 3,0.242851,0.290418
2013-07-01,Wind Farm 4,0.137524,0.182541


In [195]:
def get_figure_comparison(df, selected_zone=[]):
    if selected_zone is None or len(selected_zone) < 1:
        selected_zone = df['ZONEID'].unique()
    selected_zone = sorted(selected_zone, key=lambda x : x[-2:])
    df_zones = df[df['ZONEID'].isin(selected_zone)]
    df_zones = df_zones.groupby('DATE').mean()
    print(df_zones.info())
    print(df_zones.head())
    fig = go.Figure()
    fig.add_traces(
        go.Scatter(x=df_zones.index, y = df_zones['TARGETVAR_OBSERVED'], 
               mode = 'lines', line=dict(color='red'), name='Actual'
        )
    )
    fig.add_traces(
        go.Scatter(x=df_zones.index, y = df_zones['TARGETVAR_PREDICTED'], 
               mode = 'lines', line=dict(color='blue'), name='Predicted'
        )
    )
    fig.update_yaxes(range = [0,1])
    fig.update_layout(
        yaxis = dict(
            tickmode = 'array',
            tickvals = [0, 0.2, 0.4, 0.6, 0.8, 1],
            ticktext = ['0', '20', '40', '60', '80', '100']
        )
    )
    fig.layout.template = 'plotly_white'
    fig.update_layout( 
            title='Energy Output per day: Actual vs. Predicted',
            xaxis = dict(
                
                title='Date', 
                tickangle=45
            ),
            yaxis = dict(
                title ='Energy output in %',
            )
    )
    return fig

In [196]:
get_figure_comparison(df_comparison, ['Wind Farm 1'])

<class 'pandas.core.frame.DataFrame'>
Index: 184 entries, 2013-07-01 to 2013-12-31
Data columns (total 2 columns):
 #   Column               Non-Null Count  Dtype  
---  ------               --------------  -----  
 0   TARGETVAR_OBSERVED   184 non-null    float64
 1   TARGETVAR_PREDICTED  184 non-null    float64
dtypes: float64(2)
memory usage: 4.3+ KB
None
            TARGETVAR_OBSERVED  TARGETVAR_PREDICTED
DATE                                               
2013-07-01            0.729795             0.489216
2013-07-02            0.590002             0.466056
2013-07-03            0.751629             0.660869
2013-07-04            0.792309             0.781130
2013-07-05            0.909861             0.673151


In [197]:
df_comparison.head()

,ZONEID,TARGETVAR_OBSERVED,TARGETVAR_PREDICTED
DATE,,,
2013-07-01,Wind Farm 1,0.729795,0.489216
2013-07-01,Wind Farm 10,0.181076,0.255032
2013-07-01,Wind Farm 2,0.385629,0.296637
2013-07-01,Wind Farm 3,0.242851,0.290418
2013-07-01,Wind Farm 4,0.137524,0.182541


In [200]:
df_comparison_loaded = pd.read_csv('data_comparison_predicted_observed.csv')
df_comparison_loaded.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 1840 entries, 0 to 1839
Data columns (total 4 columns):
 #   Column               Non-Null Count  Dtype  
---  ------               --------------  -----  
 0   DATE                 1840 non-null   object 
 1   ZONEID               1840 non-null   object 
 2   TARGETVAR_OBSERVED   1840 non-null   float64
 3   TARGETVAR_PREDICTED  1840 non-null   float64
dtypes: float64(2), object(2)
memory usage: 57.6+ KB
